<a href="https://colab.research.google.com/github/ttb-folio/Pitchfork-Reviews/blob/main/SentimentAnalysistTPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!pip install transformers
!git clone https://github.com/ttb-folio/Pitchfork-Reviews.git

import numpy as np
import pandas as pd
import torch
import torch_xla
from torch.utils.data import DataLoader, Dataset
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.core.xla_model as xm
from transformers.file_utils import is_torch_tpu_available
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  34802      0 --:--:-- --:--:-- --:--:-- 34802
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
     |████████████████████████████████| 61kB 3.5MB/s 
Uninstalling torch-1.8.1+cu101:
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
  Successfully uninstalled torch-1.8.1+cu101
Uninstalling torchvision-0.9.1+cu101:
  Successfully uninstalled torchvision-0.9.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
Done updating TPU runtime
- [1 file

In [ ]:
is_torch_tpu_available()

True

In [ ]:
content_df = pd.read_csv('Pitchfork-Reviews/Data/content.csv')
review_df = pd.read_csv('Pitchfork-Reviews/Data/reviews.csv')
model_checkpoint = "distilbert-base-uncased"
text = content_df.content.fillna(' ').values
labels = review_df.score.apply(lambda x: int(x<= review_df.score.quantile(0.5))).values

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

x_train, x_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, random_state=1000)

x_train = tokenizer(list(x_train), padding = True, truncation = True)
x_test = tokenizer(list(x_test),padding = True, truncation = True)

In [ ]:
class customDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx.tolist()
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

data_train = customDataset(x_train, y_train)
data_test = customDataset(x_test, y_test)

In [ ]:
def fit(model, trainDataset, valDataset, epochs = 1, batchSize = 32, lr = 1e-3):
  device = xm.xla_device()
  model = model.to(device)
  
  trainSampler = torch.utils.data.distributed.DistributedSampler(
    data_train,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=True)
  trainLoader = DataLoader(data_train, 
                           batch_size=batchSize, 
                           sampler=trainSampler, 
                           num_workers=0, 
                           drop_last = True)

  optimizer = torch.optim.AdamW(model.parameters(), lr = lr)

  def train_loop(trainLoader):
    optimizer.zero_grad()
    for batch in trainLoader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs[0]
      loss.backward()
      xm.optimizer_step(optimizer, barrier = True)
  for epoch in range(epochs):
    model.train()
    para_loader = pl.ParallelLoader(trainLoader,[device])
    train_loop(para_loader.per_device_loader(device))
    xm.save(model.state_dict(), "model.pt")

In [ ]:
def fit_multiprocessing(rank, flags):
  fit(model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2), 
    trainDataset = data_train, valDataset = data_test, epochs = 1,  batchSize = 32)

FLAGS = {}
xmp.spawn(fit_multiprocessing, args =(FLAGS,), nprocs=8, start_method='fork')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi